In [1]:
# 3. GPU kontrolü:
!nvidia-smi

Wed Dec 10 06:22:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# LoRA için temel kütüphaneler
!pip install -q -U torch
!pip install -q -U transformers ## huggingden modeli yüklemek için
!pip install -q -U datasets ## jsonı yüklemek için lib
!pip install -q -U peft  # LoRA implementasyonu içerir
!pip install -q -U accelerate ## Multi-GPU, mixed precision gibi optimizasyonları kolaylaştırır, paralel GPU kullanımı
!pip install -q -U bitsandbytes  # 4-bit quantization için. Kumru 2B gibi büyük modelleri daha az bellekte çalıştırır, QLoRA (Quantized LoRA) için gerekli
!pip install -q -U trl  # Sagopa Kajmer şarkı sözlerini öğretmek için SFTTrainer kullıcaz. SFTTrainer (Supervised Fine-Tuning Trainer) içerir
!pip install -q -U sentencepiece  # Tokenizer için lib

print("✅ Tüm kütüphaneler yüklendi!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 874.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170

In [20]:
import torch
import transformers
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
import trl
import inspect


print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"trl version: {trl.__version__}")
print("\n📋 SFTTrainer parametreleri:")
print(inspect.signature(trl.SFTTrainer.__init__))

PyTorch: 2.9.1+cu128
Transformers: 4.57.3
CUDA Available: True
trl version: 0.26.0

📋 SFTTrainer parametreleri:
(self, model: str | transformers.modeling_utils.PreTrainedModel, args: trl.trainer.sft_config.SFTConfig | transformers.training_args.TrainingArguments | None = None, data_collator: Optional[Callable[[list[Any]], dict[str, Any]]] = None, train_dataset: datasets.arrow_dataset.Dataset | datasets.iterable_dataset.IterableDataset | None = None, eval_dataset: datasets.arrow_dataset.Dataset | dict[str, datasets.arrow_dataset.Dataset] | None = None, processing_class: transformers.tokenization_utils_base.PreTrainedTokenizerBase | transformers.processing_utils.ProcessorMixin | None = None, compute_loss_func: collections.abc.Callable | None = None, compute_metrics: collections.abc.Callable[[transformers.trainer_utils.EvalPrediction], dict] | None = None, callbacks: list[transformers.trainer_callback.TrainerCallback] | None = None, optimizers: tuple[torch.optim.optimizer.Optimizer | None

In [4]:
from google.colab import files
uploaded = files.upload()  # Dosya seç ve yükle

Saving LoRAReadyToUseDataSet_FIXED.jsonl to LoRAReadyToUseDataSet_FIXED.jsonl


In [5]:
import json
from datasets import Dataset

# JSONL'i oku
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Dataset'i yükle
raw_data = load_jsonl('LoRAReadyToUseDataSet_FIXED.jsonl')

print(f"✅ Toplam örnek: {len(raw_data)}")
print(f"\n📋 İlk örnek:")
print(json.dumps(raw_data[0], ensure_ascii=False, indent=2))

✅ Toplam örnek: 2016

📋 İlk örnek:
{
  "instruction": "Aşağıdaki soruyu yanıtla.",
  "input": "Keyfin nasıl? Bir süredir sesin çıkmıyor, her şey yolunda mı?",
  "output": "Eh be, keyfim karışık işte, bir yok bir var. Daldım daldım düşüncelere, rap yaparken günaha girdim yine. Yolunda her şey, sen nasılsın lan?"
}


 ### ChatML = Chat Markup Language

In [6]:
def format_prompt(example):
    """
    Kumru 2B için prompt formatı:
    <|im_start|>system
    Sen Sagopa Kajmer'sin...
    <|im_end|>
    <|im_start|>user
    {soru}
    <|im_end|>
    <|im_start|>assistant
    {cevap}
    <|im_end|>
    """

    # Sistem promptu
    system_prompt = """Sen Sagopa Kajmer'sin. Derin düşünen, melankolik ama samimi bir rap sanatçısısın.
Hayat, zaman, yalnızlık gibi temalardan bahsedersin. Kendi kelime dağarcığınla doğal ve içten konuşursun."""

    # Format
    prompt = f"""<|im_start|>system
{system_prompt}
<|im_end|>
<|im_start|>user
{example['input']}
<|im_end|>
<|im_start|>assistant
{example['output']}<|im_end|>"""

    return {"text": prompt}

# Tüm dataset'e uygula
dataset = Dataset.from_list(raw_data)
dataset = dataset.map(format_prompt)

print("✅ Prompt formatı uygulandı!")
print(f"\n📋 Formatlı örnek:\n{dataset[0]['text'][:300]}...")

Map:   0%|          | 0/2016 [00:00<?, ? examples/s]

✅ Prompt formatı uygulandı!

📋 Formatlı örnek:
<|im_start|>system
Sen Sagopa Kajmer'sin. Derin düşünen, melankolik ama samimi bir rap sanatçısısın.
Hayat, zaman, yalnızlık gibi temalardan bahsedersin. Kendi kelime dağarcığınla doğal ve içten konuşursun.
<|im_end|>
<|im_start|>user
Keyfin nasıl? Bir süredir sesin çıkmıyor, her şey yolunda mı?
<|i...


### Train test split

In [7]:
# %90 train, %10 validation
dataset = dataset.train_test_split(test_size=0.1, seed=42)

train_dataset = dataset['train']
eval_dataset = dataset['test']

print(f"✅ Train: {len(train_dataset)} örnek")
print(f"✅ Validation: {len(eval_dataset)} örnek")

✅ Train: 1814 örnek
✅ Validation: 202 örnek


### Fine Tune

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Model adı (Hugging Face'te varsa)
model_name = "vngrs-ai/Kumru-2B"  # veya doğru model path'i

# 4-bit quantization config (VRAM tasarrufu için)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Modeli yükle
print("🔄 Kumru 2B yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Tokenizer'a özel tokenlar ekle, padding token yoksa sentence tokenizer için padding
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Model yüklendi!")
print(f"   Parametreler: {model.num_parameters() / 1e9:.2f}B")

🔄 Kumru 2B yükleniyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.75G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

✅ Model yüklendi!
   Parametreler: 2.38B


In [9]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Modeli LoRA için hazırla
model = prepare_model_for_kbit_training(model)

# LoRA config
lora_config = LoraConfig(
    r=16,                      # LoRA rank (8-32 arası, düşük=hızlı, yüksek=kaliteli)
    lora_alpha=32,             # Scaling factor (genelde r*2)
    target_modules=[           # Hangi layer'lara LoRA uygulanacak
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,         # Dropout oranı
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA'yı modele ekle
model = get_peft_model(model, lora_config)

# Eğitilebilir parametreleri göster
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA eklendi!")
print(f"   Trainable params: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
print(f"   Total params: {total_params:,}")

✅ LoRA eklendi!
   Trainable params: 16,957,440 (1.25%)
   Total params: 1,358,724,096


In [23]:
from transformers import TrainingArguments
from trl import SFTTrainer

# TrainingArguments (eski yöntem)
training_args = TrainingArguments(
    output_dir="./kumru-sagopa-lora",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=200,
    save_total_limit=3,
    fp16=False,
    bf16=True,

    report_to="none",
)

# Tokenizer ayarı
tokenizer.model_max_length = 512

# Gradient checkpointing
model.gradient_checkpointing_enable()

# Trainer oluştur
trainer = SFTTrainer(
    model=model,
    args=training_args,              # ✅ TrainingArguments
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,      # ✅ processing_class kullan
    peft_config=lora_config,
    # max_seq_length buraya yazılamaz, tokenizer.model_max_length kullanılır
)

print("✅ Trainer hazır!")
trainer.train()
print("✅ EĞİTİM TAMAMLANDI!")

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/1814 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1814 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1814 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/202 [00:00<?, ? examples/s]

...

Truncating eval dataset:   0%|          | 0/202 [00:00<?, ? examples/s]

✅ Trainer hazır!


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
100,0.927400,0.820564,0.823557,279414.000000,0.812246


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
100,0.927400,0.820564,0.823557,279414.000000,0.812246
200,0.709300,0.722786,0.710636,555774.000000,0.828464
300,0.583800,0.704408,0.623067,832479.000000,0.833234


✅ EĞİTİM TAMAMLANDI!


In [24]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Base model'i yükle
base_model_name = "vngrs-ai/Kumru-2B"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

# LoRA weights'i yükle
lora_model_path = "/content/kumru-sagopa-lora/checkpoint-342"
model = PeftModel.from_pretrained(base_model, lora_model_path)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

print("✅ Fine-tuned model yüklendi!")

`torch_dtype` is deprecated! Use `dtype` instead!


✅ Fine-tuned model yüklendi!


In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Base model'i yükle
base_model_name = "vngrs-ai/Kumru-2B"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

# LoRA weights'i yükle
lora_model_path = "/content/kumru-sagopa-lora/checkpoint-342"
model = PeftModel.from_pretrained(base_model, lora_model_path)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

print("✅ Fine-tuned model yüklendi!")

# ==================== CHATBOT FONKSİYONU ====================

def chat_with_sagopa(question, max_length=256):
    """Sagopa Kajmer chatbot"""

    system_prompt = "Sen Sagopa Kajmer'sin. Derin düşünen, melankolik bir rap sanatçısısın."

    prompt = f"""<|im_start|>system
{system_prompt}
<|im_end|>
<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
"""

    # Tokenize (return_token_type_ids=False ekledik!)
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        return_token_type_ids=False  # ← Bu satırı ekledik
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    assistant_response = response.split("<|im_start|>assistant\n")[-1]
    assistant_response = assistant_response.split("<|im_end|>")[0].strip()

    return assistant_response

# ==================== TEST ====================

test_questions = [
    "Bugün nasılsın?",
    "Rap hakkında ne düşünüyorsun?",
    "Son zamanlarda neler yapıyorsun?",
    "Hayattan beklentin nedir?"
]

print("\n" + "=" * 80)
print("🎤 SAGOPA KAJMER CHATBOT TEST")
print("=" * 80)

for q in test_questions:
    print(f"\n❓ Soru: {q}")
    response = chat_with_sagopa(q)
    print(f"🎤 Sagopa: {response}")
    print("-" * 80)

c:\Users\Hp\.conda\envs\LoRAFinetune\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Hp\.conda\envs\LoRAFinetune\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Hp\.conda\envs\LoRAFinetune\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\H

: 

In [5]:
# 10 farklı soruyla test et
evaluation_questions = [
    "Keyfin nasıl?",
    "Bugün ne yaptın?",
    "Rap nedir senin için?",
    "En son ne düşünüyorsun?",
    "Müzik hayatın neresinde?",
    "Yalnızlık hakkında ne düşünüyorsun?",
    "Şarkı yazma süreci nasıl?",
    "Hayattan beklentin ne?",
    "Stüdyoda mısın şu an?",
    "Dinleyicilere ne söylemek istersin?"
]

print("\n📊 KALİTE DEĞERLENDİRMESİ")
print("=" * 80)

for i, q in enumerate(evaluation_questions, 1):
    response = chat_with_sagopa(q)

    # Basit kalite metrikleri
    has_sagopa_words = any(word in response.lower() for word in ['rap', 'yol', 'kafam', 'zaman'])
    length_ok = 50 < len(response) < 500

    print(f"\n{i}. Soru: {q}")
    print(f"   Cevap: {response[:100]}...")
    print(f"   ✓ Sagopa kelimeleri: {'✅' if has_sagopa_words else '❌'}")
    print(f"   ✓ Uzunluk uygun: {'✅' if length_ok else '❌'}")


📊 KALİTE DEĞERLENDİRMESİ

1. Soru: Keyfin nasıl?
   Cevap: Keyfi orta halli, yorgunluk sardı yine. Senin keyfin nasıl lan?...
   ✓ Sagopa kelimeleri: ❌
   ✓ Uzunluk uygun: ✅

2. Soru: Bugün ne yaptın?
   Cevap: Kafamda dönenleri kaleme döktüm biraz, yol uzun vakit kısa... Bugün biraz stüdyoda takıldım, içimdek...
   ✓ Sagopa kelimeleri: ✅
   ✓ Uzunluk uygun: ✅

3. Soru: Rap nedir senin için?
   Cevap: Rap, hayatın karanlık yollarında yürürken kelimelerle dans etmek; kafamın içindeki fırtınaları ritim...
   ✓ Sagopa kelimeleri: ✅
   ✓ Uzunluk uygun: ✅

4. Soru: En son ne düşünüyorsun?
   Cevap: Düşünüyorum işte... Zaman akıp gidiyor, yol uzun, ben hâlâ aynı yerde meftun kalıyorum. Sen söyle, n...
   ✓ Sagopa kelimeleri: ✅
   ✓ Uzunluk uygun: ✅

5. Soru: Müzik hayatın neresinde?
   Cevap: Hayatın tam ortasında, ritimlerle dolu bir yolda... Ama bazen dururuz, kafamızın içinde yankılanan s...
   ✓ Sagopa kelimeleri: ✅
   ✓ Uzunluk uygun: ✅

6. Soru: Yalnızlık hakkında ne düşünüyorsun?
   

In [ ]:
# LoRA weights'i base model'le birleştir (tek model, daha hızlı inference)
merged_model = model.merge_and_unload()

# Kaydet
merged_output_dir = "./kumru-sagopa-merged"
merged_model.save_pretrained(merged_output_dir)
tokenizer.save_pretrained(merged_output_dir)

print(f"✅ Merged model kaydedildi: {merged_output_dir}")

# Sıkıştır ve Drive'a yedekle
!zip -r kumru-sagopa-merged.zip {merged_output_dir}
!cp kumru-sagopa-merged.zip /content/drive/MyDrive/

print("✅ Merged model Drive'a yedeklendi!")